In [78]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [79]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim
import time
from pprint import pprint
geolocatorGoogle = GoogleV3(api_key=" ")
geolocatorNom = Nominatim(user_agent="cphDorms")

In [80]:
#Using hashlib to shorten dorm names and make them identifiable (Some Dorm Names Suck!)
import base64
from hashlib import blake2b
hasher = blake2b(digest_size=4)

def hash(input):
    hasher.update(input.encode('UTF-8'))
    return str(base64.urlsafe_b64encode(hasher.digest()).decode('UTF-8')).rstrip('=')

In [81]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)

In [82]:
soup = BeautifulSoup(results.text, "html.parser")


In [83]:
#Longitude and latitude

def coordinatesGoogle(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorGoogle.geocode(address)
             return {"latitude":location.latitude, "longitude":location.longitude}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":"NaN", "longitude":"NaN"}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

def coordinatesNomantim(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorNom.geocode(address).raw
             return {"latitude":location['lat'], "longitude":location['lon']}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":"NaN", "longitude":"NaN"}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location


print(coordinatesNomantim("Dalslandsgade 8, Copenhagen"))

{'latitude': '55.667422', 'longitude': '12.60431'}


In [84]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    print("Processing Dorm")
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")
    try:
        dorm_picture_link = container.find('img', class_="kollegiumlistimage img")['src']
    except: 
        dorm_picture_link = ""

    body = container.find('div', class_="body col-xs-12")
    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        price = room_details[3].get_text().strip()
        description = body.find('div', class_="description")

        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "uuid":hash(dorm_name + dorm_address),
        "dormPictureLink":"https://www.kollegierneskontor.dk/"+dorm_picture_link,
        "coordinates":coordinatesNomantim(dorm_address),
        "description":str(description),
        "rooms":rooms_export
    })

    print(hash(dorm_name + dorm_address))








Processing Dorm
WJmedg
Processing Dorm
Pv0aUg
Processing Dorm
CxP5aQ
Processing Dorm
fxWwjg
Processing Dorm
Couldn'd find coordinates of Emdrupvej 54 A, 2400 København NV
t1r6qw
Processing Dorm
noxiRQ
Processing Dorm
JVpoBg
Processing Dorm
XOP3tQ
Processing Dorm
9BWZqw
Processing Dorm
Couldn'd find coordinates of Holmbladsgade 70 C, 2300 København S
qf7guQ
Processing Dorm
b27RSw
Processing Dorm
KJHNAg
Processing Dorm
Couldn'd find coordinates of Kagsåkollegiet 2 , 2860 Søborg
8jMgbg
Processing Dorm
F-96Ew
Processing Dorm
8el7dQ
Processing Dorm
xToATg
Processing Dorm
QMBfFQ
Processing Dorm
A5N2IQ
Processing Dorm
ym2bsw
Processing Dorm
xTHbBQ
Processing Dorm
Couldn'd find coordinates of Robert Jacobsensvej 61 , 2300 København S
LBbSNA
Processing Dorm
PdUc3g
Processing Dorm
QdFMzw
Processing Dorm
Couldn'd find coordinates of Rektorparken 12 , 2450 København SV
FwLBiA
Processing Dorm
Couldn'd find coordinates of Stærevej 66 , 2400 København NV
8B2tmQ
Processing Dorm
E3-XLg
Processing Dorm


In [85]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)
